# LG Aimers 온라인 해커톤


목차
1. [개발 환경 및 버전 정보](#1-개발-환경-및-버전-정보)
2. [라이브러리 및 초기 세팅](#2-라이브러리-및-초기-세팅)
3. [데이터 전처리](#3-데이터-전처리)
4. [LGBM](#)
5. [CatBoost](#)
6. [NN 모델](#)
7. [앙상블](#)
8. [Submission 파일 생성](#)

## 1. 개발 환경 및 버전 정보

개발환경
|개발 환경|버전|비고|
|:---|:---:|:---|
|OS|22.04.3 LTS||
|CPU|Intel(R) Xeon(R) CPU @ 2.00GHz||
|RAM|32GB||
|GPU|P100||
|Python|3.10.12||

라이브러리
|라이브러리|버전|비고|
|:---|:---:|:---|
|jupyter|3.6.8||
|numpy|1.26.4||
|pandas|2.2.3||
|scikit-learn|1.2.2||
|scipy|1.13.1||
|lightgbm|4.5.0||
|catboost|1.2.7||
|torch|2.5.1+cu121||

## 2. 라이브러리 및 초기 세팅

In [1]:
import os
import random
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from scipy import sparse

import lightgbm as lgb
from lightgbm import early_stopping

from catboost import CatBoostClassifier, Pool

import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import OneCycleLR

In [2]:
def seed_everything(seed: int=42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    cudnn.deterministic = True
    cudnn.benchmark = False

seed_everything()

In [3]:
data_path = './data'

train_df = pd.read_csv(os.path.join(data_path, 'train.csv'), index_col= 'ID')
test_df = pd.read_csv(os.path.join(data_path, 'test.csv'), index_col= 'ID')
submission_df = pd.read_csv(os.path.join(data_path, 'sample_submission.csv'), index_col= 'ID')

## 3. 데이터 전처리

데이터 전처리의 경우 크게 2가지 방식으로 진행이 되었습니다.
1. baseline_dataset
- Baseline 코드 기반 데이터 전처리 방식에서 Ordinal Encoding 대신 One-Hot Encoding 방식으로 전처리된 데이터
- LGBM 학습에 사용

2. kiwi_dataset
- 결측값 처리와 변수 "특정 시술 유형" 및 "배아 생성 주요 이유"를 변환하고 One-Hot Encoding 방식으로 전처리된 데이터
- CatBoost 및 NN 모델에 사용

### 1) baseline_dataset

In [4]:
class BaselineDataset():
    def __init__(self):
        self._set_parameters()
        self.train = True
    
    def _set_parameters(self):
        # 전처리에 사용될 변수명
        self.CATEGORICAL_COLUMN_LIST = [
            "시술 시기 코드",
            "시술 당시 나이",
            "시술 유형",
            "특정 시술 유형",
            "배란 자극 여부",
            "배란 유도 유형",
            "단일 배아 이식 여부",
            "착상 전 유전 검사 사용 여부",
            "착상 전 유전 진단 사용 여부",
            "남성 주 불임 원인",
            "남성 부 불임 원인",
            "여성 주 불임 원인",
            "여성 부 불임 원인",
            "부부 주 불임 원인",
            "부부 부 불임 원인",
            "불명확 불임 원인",
            "불임 원인 - 난관 질환",
            "불임 원인 - 남성 요인",
            "불임 원인 - 배란 장애",
            "불임 원인 - 여성 요인",
            "불임 원인 - 자궁경부 문제",
            "불임 원인 - 자궁내막증",
            "불임 원인 - 정자 농도",
            "불임 원인 - 정자 면역학적 요인",
            "불임 원인 - 정자 운동성",
            "불임 원인 - 정자 형태",
            "배아 생성 주요 이유",
            "총 시술 횟수",
            "클리닉 내 총 시술 횟수",
            "IVF 시술 횟수",
            "DI 시술 횟수",
            "총 임신 횟수",
            "IVF 임신 횟수",
            "DI 임신 횟수",
            "총 출산 횟수",
            "IVF 출산 횟수",
            "DI 출산 횟수",
            "난자 출처",
            "정자 출처",
            "난자 기증자 나이",
            "정자 기증자 나이",
            "동결 배아 사용 여부",
            "신선 배아 사용 여부",
            "기증 배아 사용 여부",
            "대리모 여부",
            "PGD 시술 여부",
            "PGS 시술 여부"
        ]
        self.QUANTITATIVE_COLUMN_LIST = [
            "임신 시도 또는 마지막 임신 경과 연수",
            "총 생성 배아 수",
            "미세주입된 난자 수",
            "미세주입에서 생성된 배아 수",
            "이식된 배아 수",
            "미세주입 배아 이식 수",
            "저장된 배아 수",
            "미세주입 후 저장된 배아 수",
            "해동된 배아 수",
            "해동 난자 수",
            "수집된 신선 난자 수",
            "저장된 신선 난자 수",
            "혼합된 난자 수",
            "파트너 정자와 혼합된 난자 수",
            "기증자 정자와 혼합된 난자 수",
            "난자 채취 경과일",
            "난자 해동 경과일",
            "난자 혼합 경과일",
            "배아 이식 경과일",
            "배아 해동 경과일"
        ]

    def set_data(self, data:pd.DataFrame):
        '''
        전처리할 데이터를 설정하는 함수
        input:
        ---------------
        - data: 전처리할 데이터 (type: pd.DataFrame(판다스 데이터 프레임))
        '''
        self.data = data.copy()

    def set_train_mode(self, train:bool=True):
        '''
        전처리할 데이터가 학습데이터인지 아닌지를 구분하는 함수
        input:
        ---------------
        - train: 학습데이터인지 아닌지 구분 (True = 학습데이터)
        '''
        self.train = train

    def one_hot_encode(self):
        '''
        범주형 변수에 대한 One-Hot Encoding 진행
        '''
        for col in self.CATEGORICAL_COLUMN_LIST:
            self.data[col] = self.data[col].astype(str)

        if self.train:
            self.ohe = OneHotEncoder(handle_unknown='ignore')
            self.ohe.fit(self.data[self.CATEGORICAL_COLUMN_LIST])
        else:
            assert self.ohe
            
        self.encoded_cat_matrix = self.ohe.transform(self.data[self.CATEGORICAL_COLUMN_LIST])

    def get_data(self):
        '''
        전처리된 범주형, 연속형 변수들을 합쳐 데이터셋 생성
        '''
        if self.train:
            return sparse.hstack([sparse.csr_matrix(self.data[self.QUANTITATIVE_COLUMN_LIST]),
                               self.encoded_cat_matrix], format='csr'), self.data['임신 성공 여부'].values
        return sparse.hstack([sparse.csr_matrix(self.data[self.QUANTITATIVE_COLUMN_LIST]),
                               self.encoded_cat_matrix], format='csr')

### 2) kiwi_dataset

In [5]:
class KiwiDataset():
    def __init__(self):
        self._set_parameters()
        self.train = True
    
    def _set_parameters(self):
        # 전처리에 사용될 변수명
        self.TARGET_COLUMN = '임신 성공 여부'
        self.CATEGORICAL_COLUMN_LIST = ['시술 시기 코드', '시술 당시 나이', '배아 생성 주요 이유', '난자 출처', '정자 출처', '난자 기증자 나이', '정자 기증자 나이']
        self.QUANTITATIVE_COLUMN_LIST = ['임신 시도 또는 마지막 임신 경과 연수', '총 시술 횟수', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수', '총 생성 배아 수', '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '이식된 배아 수', '미세주입 배아 이식 수', '저장된 배아 수', '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수', '수집된 신선 난자 수', '저장된 신선 난자 수', '혼합된 난자 수', '파트너 정자와 혼합된 난자 수', '기증자 정자와 혼합된 난자 수', '난자 혼합 경과일', '배아 이식 경과일', '배아 해동 경과일']
        self.BINARY_COLUMN_LIST = ['배란 자극 여부', '단일 배아 이식 여부', '착상 전 유전 검사 사용 여부', '착상 전 유전 진단 사용 여부', '남성 주 불임 원인', '남성 부 불임 원인', '여성 주 불임 원인', '여성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인', '불명확 불임 원인', '불임 원인 - 난관 질환', '불임 원인 - 남성 요인', '불임 원인 - 배란 장애', '불임 원인 - 자궁경부 문제', '불임 원인 - 자궁내막증', '불임 원인 - 정자 농도', '불임 원인 - 정자 면역학적 요인', '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부', '대리모 여부', 'PGD 시술 여부', 'PGS 시술 여부']
        
        # 결측값 처리에 사용될 변수명과 처리 방식 dictionary
        self.MISSING_VALUE_RULE_DICT = {
            '특정 시술 유형': 'Unknown',
            '총 생성 배아 수': 0,
            '미세주입된 난자 수': 0,
            '미세주입에서 생성된 배아 수': 0,
            '이식된 배아 수': 0,
            '미세주입 배아 이식 수': 0,
            '저장된 배아 수': 0,
            '미세주입 후 저장된 배아 수': 0,
            '해동된 배아 수': 0,
            '해동 난자 수': 0,
            '수집된 신선 난자 수': 0,
            '저장된 신선 난자 수': 0,
            '혼합된 난자 수': 0,
            '파트너 정자와 혼합된 난자 수': 0,
            '기증자 정자와 혼합된 난자 수': 0, 
            '단일 배아 이식 여부': 0,
            '착상 전 유전 검사 사용 여부': 0,
            '착상 전 유전 진단 사용 여부': 0,
            '동결 배아 사용 여부': 0,
            '신선 배아 사용 여부': 0,
            '기증 배아 사용 여부': 0,
            '대리모 여부': 0,
            'PGD 시술 여부': 0,
            'PGS 시술 여부': 0,
        }

        # 교차 선택되어 있는 변수들의 고유값들
        ## 특정 시술 유형 변수에 대한 고유값
        self.SPECIFIC_PROCEDURE_LIST = ['IVF', 'ICSI', 'IUI', 'ICI', 'GIFT', 'FER', 'Generic DI', 'IVI', 'BLASTOCYST', 'AH', 'Unknown']
        ## 배아 생성 주요 이유 변수에 대한 고유값
        self.MAIN_PURPOSE_LIST =  '기증용, 난자 저장용, 배아 저장용, 연구용, 현재 시술용'.split(", ")

    def set_data(self, data:pd.DataFrame):
        '''
        전처리할 데이터를 설정하는 함수
        input:
        ---------------
        - data: 전처리할 데이터 (type: pd.DataFrame(판다스 데이터 프레임))
        '''
        self.data = data.copy()

    def set_train_mode(self, train:bool=True):
        '''
        전처리할 데이터가 학습데이터인지 아닌지를 구분하는 함수
        input:
        ---------------
        - train: 학습데이터인지 아닌지 구분 (True = 학습데이터)
        '''
        self.train = train

    def generate_dummy_variable(self, columns:list=['임신 시도 또는 마지막 임신 경과 연수', '난자 혼합 경과일', '배아 이식 경과일', '배아 해동 경과일']):
        '''
        결측값이 존재하는 변수들에 대한 결측값임을 나타내는 더미 변수 생성
        input:
        ---------------
        - columns: 더미 변수 생성할 결측값이 존재하는 변수명
        '''
        
        # 시술 유형이 DI에 따른 결측값 발생 변수들을 위한 더미 변수 생성
        self.data["DI 시술 여부"] = self.data["시술 유형"]=="DI"
        self.data["DI 시술 여부"] = self.data["DI 시술 여부"].astype(int)
        if self.train:
            self.BINARY_COLUMN_LIST.append("DI 시술 여부")

        # 그 외 원인 불명의 결측값 발생 변수들을 위한 더미 변수 생성
        for col in columns:
            dummy_column = f"{col} 결측 여부"
            self.data[dummy_column] = self.data[col].isnull().astype(int)
            
            if self.train:
                self.MISSING_VALUE_RULE_DICT[col] = 0
                self.BINARY_COLUMN_LIST.append(dummy_column)

    def missing_value_imputation(self, rule_dict:dict=None):
        '''
        결측값이 존재하는 변수들에 대한 결측값임을 나타내는 더미 변수 생성
        input:
        ---------------
        - rule_dict: 결측값을 처리할 변수 규칙 (dict) 없을 경우 기존에 세팅한 규칙(MISSING_VALUE_RULE_DICT)을 따름
        '''

        if rule_dict == None:
            rule_dict = self.MISSING_VALUE_RULE_DICT
        
        self.data.fillna(self.MISSING_VALUE_RULE_DICT, inplace=True)
    
    def transform_specific_procedure(self):
        '''
        변수 중 "특정 시술 유형"을 세분화 및 일반화를 통한 추가 변수 생성
        '''

        procedure_columns_list = [f"특정 시술 유형_{x}" for x in self.SPECIFIC_PROCEDURE_LIST]
        self.data[procedure_columns_list] = 0

        # 톡정 시술 복합 or 조합 여부
        self.data["복합 시술 횟수"] = 0
        self.data["세부 조합 횟수"] = 0

        for index in self.data.index:
            if type(self.data.loc[index, '특정 시술 유형']) == float:
                continue
            procedure_list = self.data.loc[index, '특정 시술 유형'].split(" / ")
            self.data.loc[index, "복합 시술 횟수"] = len(procedure_list)
            if len(procedure_list) == 0:
                continue
            for procedure in procedure_list:
                if ":" in procedure:
                    p_list = procedure.split(":")
                    # if len(p_list) != 2:
                    #     raise ValueError(f"{p_list}")
                    for p in p_list:
                        if p[0] == " ": p = p[1:]
                        if p[-1] == " ": p = p[:-1]
                    
                        self.data.loc[index, f'특정 시술 유형_{p}'] += 1
                    self.data.loc[index, '세부 조합 횟수'] += 1
                else:
                    a = procedure
                    if procedure[0] == " ": procedure = procedure[1:]
                    if procedure[-1] == " ": procedure = procedure[:-1]
                    
                    self.data.loc[index, f"특정 시술 유형_{procedure}"] += 1

        if self.train:
            self.QUANTITATIVE_COLUMN_LIST = self.QUANTITATIVE_COLUMN_LIST + procedure_columns_list
            self.QUANTITATIVE_COLUMN_LIST.append("복합 시술 횟수")
            self.QUANTITATIVE_COLUMN_LIST.append("세부 조합 횟수")
        
    
    def transform_main_purpose(self):
        '''
        변수 중 "배아 생성 주요 이유"을 세분화 및 일반화를 통한 추가 변수 생성
        '''

        purpose_columns_list = [f"배아 생성 주요 이유_{x}" for x in self.MAIN_PURPOSE_LIST]
        self.data[purpose_columns_list] = 0

        except_index_list = []
        for index in self.data.index:
            if type(self.data.loc[index, '배아 생성 주요 이유']) == float:
                except_index_list.append(index)
                continue
            purpose_list = self.data.loc[index, '배아 생성 주요 이유'].split(", ")
            # self.data.loc[index, "배아 생성 주요 이유"] = len(purpose_list)
            if len(purpose_list) == 0:
                continue
            for procedure in purpose_list:
                self.data.loc[index, f'배아 생성 주요 이유_{procedure}'] += 1

        if self.train:
            self.BINARY_COLUMN_LIST = self.BINARY_COLUMN_LIST + purpose_columns_list
            self.CATEGORICAL_COLUMN_LIST.remove("배아 생성 주요 이유")

    def one_hot_encode(self):
        '''
        범주형 변수에 대한 One-Hot Encoding 진행
        '''

        if self.train:
            self.ohe = OneHotEncoder()
            self.ohe.fit(self.data[self.CATEGORICAL_COLUMN_LIST])
        else:
            assert self.ohe
            
        self.categorical_df = pd.DataFrame(self.ohe.transform(self.data[self.CATEGORICAL_COLUMN_LIST]).toarray(), columns=self.ohe.get_feature_names_out(), index = self.data.index)

    def select_data(self):
        '''
        전처리된 변수들을 기반으로 학습 or 예측용 데이터셋 생성
        '''

        selected_column_list = self.QUANTITATIVE_COLUMN_LIST + self.BINARY_COLUMN_LIST
        transformed_df = self.data[selected_column_list]
        transformed_df
        
        for i in range(1, 11):
            transformed_df.loc[:, self.QUANTITATIVE_COLUMN_LIST[i]] = transformed_df.loc[:, self.QUANTITATIVE_COLUMN_LIST[i]].str.split("회").str[0]
        
        transformed_df = pd.concat((transformed_df, self.categorical_df), axis=1)
        self.transformed_df = transformed_df.astype(int)

    def get_data(self):
        if self.train:
            return self.transformed_df, self.data[self.TARGET_COLUMN]
    
        else:
            return self.transformed_df

## 4. LGBM

### 1) LGBM 데이터 - baseline_dataset

In [6]:
baseline_dataset = BaselineDataset()
baseline_dataset.set_data(train_df)
baseline_dataset.one_hot_encode()
X_lgbm, y_lgbm = baseline_dataset.get_data()

In [7]:
baseline_dataset.set_train_mode(False)
baseline_dataset.set_data(test_df)
baseline_dataset.one_hot_encode()
X_test_lgbm = baseline_dataset.get_data()

### 2) 파라미터 최적화

※ 해당 부분은 최적 파라미터를 추출하는 코드이므로 주석 처리 되었습니다.

In [8]:
# import lightgbm as lgb
# from sklearn.model_selection import train_test_split

# # 8:2 비율로 훈련 데이터, 검증 데이터 분리 (베이지안 최적화 수행용)
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
#                                                       test_size=0.2, 
#                                                       random_state=0)

# # 베이지안 최적화용 데이터셋
# bayes_dtrain = lgb.Dataset(X_train, y_train)
# bayes_dvalid = lgb.Dataset(X_valid, y_valid)

In [9]:
# # 베이지안 최적화를 위한 하이퍼파라미터 범위
# param_bounds = {
#     'num_leaves': (30, 100),  
#     'lambda_l1': (0, 1),  
#     'lambda_l2': (0, 2),  
#     'feature_fraction': (0.7, 1),  
#     'bagging_fraction': (0.5, 0.8),  
#     'min_child_samples': (5, 50),  
#     'min_child_weight': (25, 50)  
# }

# # 값이 고정된 하이퍼파라미터
# fixed_params_lgbm = {'objective': 'binary', # binary classification
#                 'learning_rate': 0.005, # 0.01~0.001
#                 'bagging_freq': 1, # 0 or 1
#                 'force_row_wise': True,
#                 'random_state': 1991}

In [10]:
# from sklearn.metrics import roc_auc_score
# from lightgbm import early_stopping

# def eval_function(num_leaves, lambda_l1, lambda_l2, feature_fraction,
#                   bagging_fraction, min_child_samples, min_child_weight):
#     '''최적화하려는 평가지표 계산 함수'''
    
#     # 베이지안 최적화를 수행할 하이퍼파라미터 
#     params = {'num_leaves': int(round(num_leaves)),
#               'lambda_l1': lambda_l1,
#               'lambda_l2': lambda_l2,
#               'feature_fraction': feature_fraction,
#               'bagging_fraction': bagging_fraction,
#               'min_child_samples': int(round(min_child_samples)),
#               'min_child_weight': min_child_weight,
#               'feature_pre_filter': False}
#     # 고정된 하이퍼파라미터도 추가
#     params.update(fixed_params_lgbm)
    
#     print('하이퍼파라미터:', params)    
    
#     # LightGBM 모델 훈련
#     lgb_model = lgb.train(params=params, 
#                            train_set=bayes_dtrain,
#                            num_boost_round=2500,
#                            valid_sets=bayes_dvalid,
#                            callbacks=[early_stopping(stopping_rounds=200)])
#     # 검증 데이터로 예측 수행
#     preds = lgb_model.predict(X_valid) 
#     # roc-auc 계산
#     roc_auc = roc_auc_score(y_valid, preds)
#     print(f'roc-auc : {roc_auc}\n')
    
#     return roc_auc

In [11]:
# from bayes_opt import BayesianOptimization

# # 베이지안 최적화 객체 생성
# optimizer = BayesianOptimization(f=eval_function,      # 평가지표 계산 함수
#                                  pbounds=param_bounds, # 하이퍼파라미터 범위
#                                  random_state=0)

In [12]:
# # 베이지안 최적화 수행
# optimizer.maximize(init_points=15, n_iter=60)

In [13]:
# # 평가함수 점수가 최대일 때 하이퍼파라미터
# max_params_lgbm = optimizer.max['params']

In [14]:
# # 정수형 하이퍼파라미터 변환
# max_params_lgbm['num_leaves'] = int(round(max_params_lgbm['num_leaves']))
# max_params_lgbm['min_child_samples'] = int(round(max_params_lgbm['min_child_samples']))

# # 값이 고정된 하이퍼파라미터 추가
# max_params_lgbm.update(fixed_params_lgbm)

In [15]:
# max_params_lgbm

### 3) 층화 K-Fold 모델 학습 및 테스트 데이터 예측

In [16]:
# lgbm roc auc score 계산 함수
def lgb_roc_auc(y_pred, dataset):
    y_true = dataset.get_label()
    return "roc_auc", roc_auc_score(y_true, y_pred), True  # (지표 이름, 값, 높은 값이 더 좋은지 여부)

In [17]:
max_params_lgbm = {
	'bagging_fraction': 0.5863913416418778,
	'feature_fraction': 0.9581372279421587,
	'lambda_l1': 0.3465698371871525,
	'lambda_l2': 1.839362361400739,
	'min_child_samples': 49,
	'min_child_weight': 41.152903682697854,
	'num_leaves': 31,
	'objective': 'binary',
	'learning_rate': 0.005,
	'bagging_freq': 1,
	'force_row_wise': True,
	'random_state': 1991
 }

In [18]:
# 층화 K 폴드 교차 검증기
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=1991)

In [19]:
# OOF 방식으로 훈련된 모델로 검증 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_val_preds_lgbm = np.zeros(X_lgbm.shape[0]) 
# OOF 방식으로 훈련된 모델로 테스트 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_test_preds_lgbm = np.zeros(X_test_lgbm.shape[0])

In [20]:
# OOF 방식으로 모델 훈련, 검증, 예측
for idx, (train_idx, valid_idx) in enumerate(folds.split(X_lgbm, y_lgbm)):
    # 각 폴드를 구분하는 문구 출력
    print('#'*40, f'폴드 {idx+1} / 폴드 {folds.n_splits}', '#'*40)

    # 데이터가 DataFrame인지 확인 후 인덱싱 방식 결정
    if isinstance(X_lgbm, pd.DataFrame):
        X_train, X_valid = X_lgbm.iloc[train_idx], X_lgbm.iloc[valid_idx]
    else:  # numpy.ndarray일 경우
        X_train, X_valid = X_lgbm[train_idx], X_lgbm[valid_idx]

    y_train, y_valid = y_lgbm[train_idx], y_lgbm[valid_idx]

    # LightGBM 전용 데이터셋 생성
    dtrain = lgb.Dataset(X_train, y_train) # LightGBM 전용 훈련 데이터셋
    dvalid = lgb.Dataset(X_valid, y_valid) # LightGBM 전용 검증 데이터셋
                          
    # LightGBM 모델 훈련
    lgb_model = lgb.train(params=max_params_lgbm,    # 최적 하이퍼파라미터
                          train_set=dtrain,     # 훈련 데이터셋
                          num_boost_round=2500, # 부스팅 반복 횟수
                          valid_sets=dvalid,    # 성능 평가용 검증 데이터셋
                          feval = lgb_roc_auc,
                          callbacks=[early_stopping(stopping_rounds=200)])
    
    # 테스트 데이터를 활용해 OOF 예측
    oof_test_preds_lgbm += lgb_model.predict(X_test_lgbm)/folds.n_splits
    # 모델 성능 평가를 위한 검증 데이터 타깃값 예측 
    oof_val_preds_lgbm[valid_idx] += lgb_model.predict(X_valid)
    
    # 검증 데이터 예측 확률에 대한 ROC-AUC
    roc_auc = roc_auc_score(y_valid, oof_val_preds_lgbm[valid_idx])
    print(f'폴드 {idx+1} roc-auc : {roc_auc}\n')

######################################## 폴드 1 / 폴드 10 ########################################
[LightGBM] [Info] Number of positive: 59605, number of negative: 171110
[LightGBM] [Info] Total Bins 883
[LightGBM] [Info] Number of data points in the train set: 230715, number of used features: 187
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258349 -> initscore=-1.054567
[LightGBM] [Info] Start training from score -1.054567
Training until validation scores don't improve for 200 rounds
Did not meet early stopping. Best iteration is:
[2389]	valid_0's binary_logloss: 0.487666	valid_0's roc_auc: 0.740225
폴드 1 roc-auc : 0.7402252385799368

######################################## 폴드 2 / 폴드 10 ########################################
[LightGBM] [Info] Number of positive: 59606, number of negative: 171110
[LightGBM] [Info] Total Bins 884
[LightGBM] [Info] Number of data points in the train set: 230716, number of used features: 187
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258352 -> i

### 4) 모델 평가

In [21]:
print('LGBM OOF 검증 데이터 ROC-AUC :', roc_auc_score(y_lgbm, oof_val_preds_lgbm))

LGBM OOF 검증 데이터 ROC-AUC : 0.7404986919647052


## 5. CatBoost

### 1) CatBoost 데이터 - kiwi_dataset

In [22]:
kiwi_dataset = KiwiDataset()
kiwi_dataset.set_data(train_df)
kiwi_dataset.generate_dummy_variable()
kiwi_dataset.missing_value_imputation()
kiwi_dataset.transform_specific_procedure()
kiwi_dataset.transform_main_purpose()
kiwi_dataset.one_hot_encode()
kiwi_dataset.select_data()
X_cat, y_cat = kiwi_dataset.get_data()

In [23]:
kiwi_dataset.set_train_mode(False)
kiwi_dataset.set_data(test_df)
kiwi_dataset.generate_dummy_variable()
kiwi_dataset.missing_value_imputation()
kiwi_dataset.transform_specific_procedure()
kiwi_dataset.transform_main_purpose()
kiwi_dataset.one_hot_encode()
kiwi_dataset.select_data()

X_test_cat = kiwi_dataset.get_data()

### 2) 파라미터 최적화

※ 해당 부분은 최적 파라미터를 추출하는 코드이므로 주석 처리 되었습니다.

In [24]:
# from catboost import CatBoostClassifier, Pool
# from sklearn.model_selection import train_test_split

# # 8:2 비율로 훈련 데이터, 검증 데이터 분리 (베이지안 최적화 수행용)
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
#                                                       test_size=0.2, 
#                                                       random_state=0)

# # CatBoost 전용 데이터셋
# bayes_dtrain = Pool(X_train, y_train, cat_features=None)
# bayes_dvalid = Pool(X_valid, y_valid, cat_features=None)

In [25]:
# # 베이지안 최적화를 위한 하이퍼파라미터 범위
# param_bounds = {
#     'depth': (6, 10),
#     'learning_rate': (0.05, 0.1),
#     'l2_leaf_reg': (8, 10),
#     'bagging_temperature': (0.5, 1.0),
#     'border_count': (200, 255)
# }

# # 고정된 하이퍼파라미터
# fixed_params_cat = {
#     'loss_function': 'Logloss',
#     'eval_metric': 'AUC',
#     'iterations': 2500,
#     'random_seed': 1991,
#     'verbose': 200
# }

In [26]:
# from sklearn.metrics import roc_auc_score

# # 평가 함수 정의
# def eval_function(depth, learning_rate, l2_leaf_reg, bagging_temperature, border_count):
#     params = {
#         'depth': int(round(depth)),
#         'learning_rate': learning_rate,
#         'l2_leaf_reg': l2_leaf_reg,
#         'bagging_temperature': bagging_temperature,
#         'border_count': int(round(border_count))
#     }
    
#     params.update(fixed_params_cat)
#     print('하이퍼파라미터:', params)
    
#     # CatBoost 모델 훈련
#     cat_model = CatBoostClassifier(**params)
#     cat_model.fit(bayes_dtrain, 
#                   eval_set=bayes_dvalid, 
#                   early_stopping_rounds=200, 
#                   verbose=0)
    
#     # 검증 데이터 예측
#     preds = cat_model.predict_proba(X_valid)[:, 1]
    
#     # ROC-AUC 계산
#     roc_auc = roc_auc_score(y_valid, preds)
#     print(f'ROC-AUC : {roc_auc}\n')
    
#     return roc_auc

In [27]:
# from bayes_opt import BayesianOptimization

# # 베이지안 최적화 객체 생성
# optimizer = BayesianOptimization(f=eval_function,      # 평가지표 계산 함수
#                                  pbounds=param_bounds, # 하이퍼파라미터 범위
#                                  random_state=0)

In [28]:
# # 베이지안 최적화 수행
# optimizer.maximize(init_points=15, n_iter=60)

In [29]:
# max_params_cat = optimizer.max['params']
# max_params_cat['depth'] = int(round(max_params_cat['depth']))
# max_params_cat['border_count'] = int(round(max_params_cat['border_count']))
# max_params_cat.update(fixed_params_cat)

In [30]:
# max_params_cat

### 3) 층화 K-Fold 모델 학습 및 테스트 데이터 예측

In [31]:
max_params_cat = {
    'bagging_temperature': 0.6322778060523135,
    'border_count': 243,
    'depth': 8,
    'l2_leaf_reg': 9.136867897737297,
    'learning_rate': 0.05093949002181776,
    'loss_function': 'Logloss',
    'eval_metric': 'AUC',
    'iterations': 2500,
    'random_seed': 1991,
    'verbose': 200
 }

In [32]:
# 층화 K 폴드 교차 검증기
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=1991)

In [33]:
oof_val_preds_cat = np.zeros(X_cat.shape[0]) 
oof_test_preds_cat = np.zeros(X_test_cat.shape[0]) 

In [34]:
for idx, (train_idx, valid_idx) in enumerate(folds.split(X_cat, y_cat)):
    print('#'*40, f'폴드 {idx+1} / 폴드 {folds.n_splits}', '#'*40)
    
    # 데이터가 DataFrame인지 확인 후 인덱싱 방식 결정
    if isinstance(X_cat, pd.DataFrame):
        X_train, X_valid = X_cat.iloc[train_idx], X_cat.iloc[valid_idx]
    else:  # numpy.ndarray일 경우
        X_train, X_valid = X_cat[train_idx], X_cat[valid_idx]

    y_train, y_valid = y_cat[train_idx], y_cat[valid_idx]
    
    dtrain = Pool(X_train, y_train, cat_features=None)
    dvalid = Pool(X_valid, y_valid, cat_features=None)
    
    cat_model = CatBoostClassifier(**max_params_cat)
    cat_model.fit(dtrain, eval_set=dvalid, early_stopping_rounds=200, verbose=500)
    
    # 테스트 데이터 예측
    oof_test_preds_cat += cat_model.predict_proba(X_test_cat)[:, 1] / folds.n_splits
    # 검증 데이터 예측
    oof_val_preds_cat[valid_idx] += cat_model.predict_proba(X_valid)[:, 1]
    
    # 검증 데이터 ROC-AUC 계산
    roc_auc = roc_auc_score(y_valid, oof_val_preds_cat[valid_idx])
    print(f'폴드 {idx+1} ROC-AUC : {roc_auc}\n')

######################################## 폴드 1 / 폴드 10 ########################################


C:\Users\jun04\AppData\Local\Temp\ipykernel_32984\1699458433.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_train, y_valid = y_cat[train_idx], y_cat[valid_idx]


0:	test: 0.7076247	best: 0.7076247 (0)	total: 166ms	remaining: 6m 54s
500:	test: 0.7400435	best: 0.7400682 (464)	total: 7.36s	remaining: 29.4s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.7401129161
bestIteration = 512

Shrink model to first 513 iterations.
폴드 1 ROC-AUC : 0.7401129160583952

######################################## 폴드 2 / 폴드 10 ########################################


C:\Users\jun04\AppData\Local\Temp\ipykernel_32984\1699458433.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_train, y_valid = y_cat[train_idx], y_cat[valid_idx]


0:	test: 0.7164902	best: 0.7164902 (0)	total: 17.3ms	remaining: 43.3s
500:	test: 0.7390021	best: 0.7391402 (418)	total: 7.51s	remaining: 30s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.7391402254
bestIteration = 418

Shrink model to first 419 iterations.
폴드 2 ROC-AUC : 0.7391402253617091

######################################## 폴드 3 / 폴드 10 ########################################
0:	test: 0.7041943	best: 0.7041943 (0)	total: 16.8ms	remaining: 42s


...

500:	test: 0.7404301	best: 0.7405930 (470)	total: 8.23s	remaining: 32.8s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.7405929507
bestIteration = 470

Shrink model to first 471 iterations.
폴드 8 ROC-AUC : 0.7405929506794646

######################################## 폴드 9 / 폴드 10 ########################################


C:\Users\jun04\AppData\Local\Temp\ipykernel_32984\1699458433.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_train, y_valid = y_cat[train_idx], y_cat[valid_idx]


0:	test: 0.7286846	best: 0.7286846 (0)	total: 18.7ms	remaining: 46.7s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.744791603
bestIteration = 287

Shrink model to first 288 iterations.
폴드 9 ROC-AUC : 0.7447916029670334

######################################## 폴드 10 / 폴드 10 ########################################


C:\Users\jun04\AppData\Local\Temp\ipykernel_32984\1699458433.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_train, y_valid = y_cat[train_idx], y_cat[valid_idx]


0:	test: 0.7225677	best: 0.7225677 (0)	total: 18.2ms	remaining: 45.6s
500:	test: 0.7407621	best: 0.7409408 (423)	total: 8.26s	remaining: 33s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.7409407566
bestIteration = 423

Shrink model to first 424 iterations.
폴드 10 ROC-AUC : 0.7409407566329922



### 4) 모델 평가

In [35]:
print('CatBoost OOF 검증 데이터 ROC-AUC :', roc_auc_score(y_cat, oof_val_preds_cat))

CatBoost OOF 검증 데이터 ROC-AUC : 0.7399695623163609


## 6. Neural Network

### 1) NN 모델 데이터 - kiwi_dataset

In [36]:
kiwi_dataset = KiwiDataset()
kiwi_dataset.set_data(train_df)
kiwi_dataset.generate_dummy_variable()
kiwi_dataset.missing_value_imputation()
kiwi_dataset.transform_specific_procedure()
kiwi_dataset.transform_main_purpose()
kiwi_dataset.one_hot_encode()
kiwi_dataset.select_data()
X_nn, y_nn = kiwi_dataset.get_data()

In [37]:
kiwi_dataset.set_train_mode(False)
kiwi_dataset.set_data(test_df)
kiwi_dataset.generate_dummy_variable()
kiwi_dataset.missing_value_imputation()
kiwi_dataset.transform_specific_procedure()
kiwi_dataset.transform_main_purpose()
kiwi_dataset.one_hot_encode()
kiwi_dataset.select_data()

X_test_nn = kiwi_dataset.get_data()

In [38]:
# 데이터 변환
X_tensor = np.array(X_nn) 
y_tensor = np.array(y_nn).astype(int)

X_test_tensor = np.array(X_test_nn)

In [39]:
# PyTorch Dataset 생성
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, X, y=None):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32) if y is not None else None

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.y is not None:
            return self.X[idx], self.y[idx]
        return self.X[idx]

### 2) 모델 구축

In [40]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim):
        super(NeuralNetwork, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),  # Dropout 증가
            
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            
            nn.Linear(128, 1),  # 이진 분류: 출력층은 1개 뉴런
            nn.Sigmoid()        # 이진 분류 확률 출력
        )
        
    def forward(self, x):  
        return self.model(x)

### 3) 층화 K-Fold 모델 학습 및 테스트 데이터 예측

In [41]:
# GPU 사용 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [42]:
# K-Fold 설정
n_splits = 10
folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=1991)

In [43]:
# OOF 예측 저장용
oof_val_preds_nn = np.zeros(X_tensor.shape[0])
oof_test_preds_nn = np.zeros(X_test_tensor.shape[0])

In [44]:
# K-Fold 훈련
for fold_idx, (train_idx, valid_idx) in enumerate(folds.split(X_tensor, y_tensor)):
    print(f"\n#### Fold {fold_idx+1}/{n_splits} ####")

    # 훈련/검증 데이터 설정
    X_train, y_train = X_tensor[train_idx], y_tensor[train_idx]
    X_valid, y_valid = X_tensor[valid_idx], y_tensor[valid_idx]

    # NaN 값 0으로 채우기 (필요하면 평균값으로 대체 가능)
    X_train = np.nan_to_num(X_train, nan=0.0)
    X_valid = np.nan_to_num(X_valid, nan=0.0)

    train_dataset = CustomDataset(X_train, y_train)
    valid_dataset = CustomDataset(X_valid, y_valid)
    
    train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=1024, shuffle=False)

    # 모델 초기화
    model = NeuralNetwork(input_dim=X_train.shape[1]).to(device)

    # 최적화 알고리즘 & 스케줄러 설정
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)

    # `total_steps`를 정확하게 설정해야 ValueError 방지 가능!
    total_steps = len(train_loader) * 1000
    scheduler = OneCycleLR(optimizer, max_lr=0.003, total_steps=total_steps, pct_start=0.1)

    criterion = nn.BCELoss()  
    best_auc = 0.0  # AUC 기준 Early Stopping

    # Early Stopping 설정
    best_valid_loss = np.inf
    patience, patience_counter = 20, 0

    # 학습 진행
    for epoch in range(1000):
        model.train()
        train_loss = 0.0

        for X_batch, y_batch in train_loader:  
            X_batch, y_batch = X_batch.to(device), y_batch.to(device).unsqueeze(1)
    
            optimizer.zero_grad()
            preds = model(X_batch)
    
            loss = criterion(preds, y_batch)  # Loss 계산
            loss.backward()  # 역전파
            optimizer.step()  # 가중치 업데이트
    
            train_loss += loss.item()
            scheduler.step() # lr update

        # 검증 데이터 평가
        model.eval()
        valid_loss, valid_preds = 0.0, []
        with torch.no_grad():
            for X_batch, y_batch in valid_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device).unsqueeze(1)
                preds = model(X_batch)
                loss = criterion(preds, y_batch)
                valid_loss += loss.item()
                valid_preds.extend(preds.cpu().numpy())

        valid_loss /= len(valid_loader)
        train_loss /= len(train_loader)
        roc_auc = roc_auc_score(y_valid, np.array(valid_preds).flatten())

        # 10 에폭마다 출력 (첫 에폭 포함)
        if (epoch + 1) % 10 == 0 or epoch == 0:
            current_lr = optimizer.param_groups[0]['lr']  # 현재 Learning Rate 확인
            print(f"Epoch {epoch+1}: Train Loss = {train_loss}, Valid Loss = {valid_loss:}, AUC = {roc_auc}, LR = {current_lr}")

        # AUC & Loss 기준 Best Model 저장
        if roc_auc > best_auc or valid_loss < best_valid_loss:
            best_auc = max(best_auc, roc_auc)
            best_valid_loss = min(best_valid_loss, valid_loss)
            patience_counter = 0
            best_model_state = model.state_dict()
            best_iteration = epoch + 1  # Best Iteration 저장
        else:
            patience_counter += 1

        # Early Stopping
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            print(f"Best iteration: [{best_iteration}]  Valid Loss: {best_valid_loss}  AUC: {best_auc}")
            break

    # 최적 모델 로드
    model.load_state_dict(best_model_state)

    # 검증 데이터 예측 저장
    model.eval()
    valid_preds = []
    with torch.no_grad():
        for X_batch in valid_loader:
            X_batch = X_batch[0].to(device)  
            preds = model(X_batch)
            valid_preds.extend(preds.cpu().numpy())

    valid_preds = np.array(valid_preds).flatten()
    
    print(f"Fold {fold_idx+1} Valid Preds Min/Max: {valid_preds.min()}, {valid_preds.max()}")
    print(f"Fold {fold_idx+1} Valid AUC: {roc_auc_score(y_valid, valid_preds)}")

    # OOF 예측 저장
    oof_val_preds_nn[valid_idx] = valid_preds

    # 테스트 데이터 예측 저장
    test_dataset = CustomDataset(X_test_tensor)
    test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)

    test_preds = []
    with torch.no_grad():
        for X_batch in test_loader:
            X_batch = X_batch.to(device)
            preds = model(X_batch)
            test_preds.extend(preds.cpu().numpy())

    oof_test_preds_nn += np.array(test_preds).flatten() / n_splits


#### Fold 1/10 ####
Epoch 1: Train Loss = 0.5546261921393133, Valid Loss = 0.5241704743642074, AUC = 0.706672419172276, LR = 0.00012071061595315047
Epoch 10: Train Loss = 0.49386398784354724, Valid Loss = 0.4913685115484091, AUC = 0.7329642435181809, LR = 0.0001904848026045092
Epoch 20: Train Loss = 0.4903081851986657, Valid Loss = 0.48922450038102955, AUC = 0.7360884518891964, LR = 0.0003950390612538755
Epoch 30: Train Loss = 0.4889277734060203, Valid Loss = 0.48879584670066833, AUC = 0.7372055582907786, LR = 0.000713637822416415
Epoch 40: Train Loss = 0.4883318481455862, Valid Loss = 0.48901418309945327, AUC = 0.7371396966651845, LR = 0.0011150916822163179
Epoch 50: Train Loss = 0.48777688077065795, Valid Loss = 0.4883568493219522, AUC = 0.7375974323821239, LR = 0.0015601000905663418
Epoch 60: Train Loss = 0.4870769601743833, Valid Loss = 0.4887784249507464, AUC = 0.7374017732838675, LR = 0.0020050987004944063
Early stopping at epoch 66
Best iteration: [46]  Valid Loss: 0.4876569475

### 4) 모델 평가

In [45]:
print('NN OOF 검증 데이터 ROC-AUC :', roc_auc_score(y_tensor, oof_val_preds_nn))

NN OOF 검증 데이터 ROC-AUC : 0.7371480163680295


## 7. 앙상블

### 1) 앙상블 모델 평가

In [48]:
ensemble_val_preds = oof_val_preds_lgbm * 0.6 + oof_val_preds_cat * 0.25 + oof_val_preds_nn * 0.15

print('앙상블 OOF 검증 데이터 ROC-AUC :',  roc_auc_score(y_cat, ensemble_val_preds))

앙상블 OOF 검증 데이터 ROC-AUC : 0.7407225517194111


### 2) 테스트 데이터 앙상블 예측

In [49]:
ensemble_test_preds = oof_test_preds_lgbm * 0.6 + oof_test_preds_cat * 0.25 + oof_test_preds_nn * 0.15

## 8. 최종 결과 제출

In [50]:
submission_df["probability"] = ensemble_test_preds
submission_df.to_csv("lgbm_cat_nn_submission.csv", index=True)